In [1]:
import re
import os
import pandas as pd
import rpy2.robjects as ro
from rpy2.robjects import r as R
from rpy2.robjects import rinterface
from rpy2.robjects import pandas2ri
import numpy as np

In [2]:
gating_file = "/ifs/projects/proj052/pipeline_proj052/gating_design.tsv"

In [3]:
gate_df = pd.read_table(gating_file, sep="\t", index_col=None)
gate_dict = {"P": "polygon",
             "Q": "quadrant",
             "R": "rectangle",
             "G": "range"}

In [24]:
np.any(gate_df["panels"])

KeyError: 'panels'

In [4]:
regex = r"\S\("

In [5]:
print gate_df.ix[1]["gating"]
gates = re.findall(regex, gate_df.ix[1]["gating"])
markers = re.split(regex, gate_df.ix[1]["gating"])
markers.remove('')
print gates, markers

P(lymph)Q(CD3+CD8+)R(CD45R+CCR7+)
['P(', 'Q(', 'R('] ['lymph)', 'CD3+CD8+)', 'CD45R+CCR7+)']


In [6]:
if len(gates) == len(markers):
    for i,j in enumerate(gates):
        g = j.strip("(")
        gate = gate_dict[g]
        dimensions = (markers[i]).strip(")")
        print gate, dimensions

polygon lymph
quadrant CD3+CD8+
rectangle CD45R+CCR7+


In [7]:
fcs_files = "/ifs/projects/proj052/pipeline_proj052/fcs.dir/ZZF4/"
panel = "P1"
comp_matrix = "/ifs/projects/proj052/pipeline_proj052/comp_matrices.dir/P1-compensation_matrix.txt"

In [8]:
# set up flow files first
R('''suppressPackageStartupMessages(library(flowCore))''')
R('''suppressPackageStartupMessages(library(flowWorkspace))''')
R('''source("/ifs/projects/proj052/src/R_scripts/FlowProcess.R")''')
R('''Sys.setlocale('LC_ALL', 'C')''')

R('''path.to.files <- "%s"''' % fcs_files)
R('''flowdata <- read.flowSet(path=path.to.files, transformation=F,'''
  '''patern="%s")''' % panel)
R('''gc()''')

R('''twin.split <- strsplit(x=sampleNames(flowdata), split=" ", fixed=T)''')
#R('''twin.ids = unlist(lapply(twin.split, FUN=function(x) {paste("Twin", '''
#  '''x[1], x[2], sep="_")}))''')
#R('''sampleNames(flowdata) <- twin.ids''')

R('''flow.data <- set_marker_id(flowdata)''')
R('''colnames(flow.data) <- gsub("-", ".", colnames(flow.data))''')

<StrVector - Python:0x2b12ee3deb00 / R:0x48b0ee60>
[str, str, str, ..., str, str, str]

In [9]:
# compensation matrix
R('''comp <- read.table("%s", h=T, row.names=1)''' % comp_matrix)
R('''fs_comp <- compensate(x=flow.data, spillover=comp)''')
R('''gc()''')

<Matrix - Python:0x2b12ee3fbe60 / R:0x3ac3fe60>
[1398308.000000, 1016438191.000000, 74.700000, ..., 1064982918.000000, 112.500000, 8125.200000]

In [11]:
# set up gating strategy
R('''gs <- GatingSet(fs_comp)''')
R('''print(gs)''')
#R('''biexpTrans <- transformList(colnames(Data(flowJoTrans()''')

/ifs/apps/apps/python-2.7.9/lib/python2.7/site-packages/rpy2-2.6.1-py2.7-linux-x86_64.egg/rpy2/robjects/functions.py:106: UserWarning: Error: cannot allocate vector of size 61.4 Mb

  res = super(Function, self).__call__(*new_args, **new_kwargs)


RRuntimeError: Error: cannot allocate vector of size 61.4 Mb


In [28]:
print 4.5//2

2.0
